## Introduction
In this notebook, we are attempting to simulate the signals that our visual cortex receives from our eyes by convolving an *edge operator* across an image. We will simulate this on an image of a natural scene and also a computer generated image.

### Images
We will be applying our edge operators to two images: `natural_img` a natural image, taken by a camera, of the world and a `synthetic_img` which will be an image that was not captured by a camera, but created on a computer. Before we begin applying these operators to our images, we need to accomplish two prerequisite tasks:
1. Read the images into the program
2. Build our edge operators

In the following code block we will accomplish the first of these tasks. The second task will be the focus of the remainder of this assignment. Let's read those images into our program and display them so that we know what we are working with.

In [ ]:
from PIL import Image

# Read Images from Disk
natural_img = Image.open("./images/natural_image_ED.jpg")      # natural image
synthetic_img = Image.open("./images/synthetic_image_ED.jpg")  # synthetic image

# Display Images
print("Natural Image")
display(natural_img)
print("Synthetic Image")
display(synthetic_img)

## Building our Edge Operators
Edge operators generally have a few common features. The first of which is that the sum of weights in our operator should be zero. Naturally this brings us to a second feature, some weights must be negative. Those weights that are negative are called *inhibitory* and act to supress the magnitude of the signal sent to our brain. Those weights that are positive are called *excitatory*, they act to increase the magnitude of the signal sent to our brain. Additionally, weights in our operators can have zero value. Zero value weights do not contribute to the magnitude of the signal sent to the brain, but can influence the scene that our operator responds to.

For example, a simple edge detector made with the intention of recognizing simple falling edges might have an archetecture in which the inhibitory -negatively weighted- region is on the right hand side of the operator and the excitatory on the left:
$$
\begin{bmatrix}
1    &0    &-1\\
1    &0    &-1\\
1    &0    &-1\\
\end{bmatrix}
$$
>When the signal generated by a simple cell like this is negative, the signal is zero.

Moreover, we can create the rising edge operator by multiplying each element by $-1$; effectively swapping the placement of the excitatory and inhibitory regions. But this is not necessary because we are using computers and not cells, we can take the absolute value of the signal produced by our operator. This is an advantage because now the operator displayed above can detect both rising and falling verticle edges. This also saves us from having to convolve both operators across the image.

Before we can convolve our first operator we need to define a threshold. Almost every step in the convolution will result in a value, it is our job to decide at which signal strength we wish to declare that an edge exists. If we choose a higher value, our operator will find a smaller number of edges, but the edges it does find will be well pronounced. If our threshold is small, then our operator will find many edges-- even where they don't exist. Despite having these general rules, we don't have a law describing how we should choose a threshold.

It seems sensible to me to set the threshold to the average pixel value in the image. So lets find that threshold for each image.

In [ ]:
# Image dimensions and threshold initialization
natural_img_cols, natural_img_rows = natural_img.size        # natural image dimensions
synthetic_img_cols, synthetic_img_rows = synthetic_img.size  # synthetic image dimensions

natural_threshold = 0
synthetic_threshold = 0

# Find threshold value for natural image
pixels_in_img = natural_img_cols * natural_img_rows
pixel_sum = 0

for u in range(natural_img_cols - 1):
    for v in range(natural_img_rows -1):
        pixel_sum += natural_img.getpixel((u, v))[0]
        
natural_img_threshold = int(pixel_sum/pixels_in_img)

# Find threshold value for synthetic image
pixels_in_img = synthetic_img_cols * synthetic_img_rows
pixel_sum = 0

for u in range(synthetic_img_cols - 1):
    for v in range(synthetic_img_rows -1):
        pixel_sum += synthetic_img.getpixel((u, v))[0]
        
synthetic_img_threshold = int(pixel_sum/pixels_in_img)

print(f"Natural Image Threshold: {natural_img_threshold}")
print(f"Synthetic Image Threshold: {synthetic_img_threshold}")

Viewing the natural image and synthetic image we notice that they are dark and light; respectively. Thus it is no suprise that the darker image of a gray scale cathedral on a stormy day has a lower average pixel value than the mostly white synthetic image. Hence why we did not choose the same threshold for both images.

Now, lets write a function to aid us in our convolutions. We should take special consideration to ensure that this function will work for any $n \times n$ operator such that $n$ is an odd number greater than zero.

In [ ]:
def flatten_operator(op):
    flat_op = []
    for row in op:
        for element in row:
            flat_op.append(element)
    return flat_op

def get_window(x, y, size, img):
    """Given an x, y position and a size, this function will return the pixel values of the subregion of the image
    where x, y is the top left pixel in the size by size subregion."""
    window = []
    for i in range(size):
        for j in range(size):
            window.append(img.getpixel((x+i, y+j))[0])
    return window

def convolve_operator(operator, image, edge_image=Image.new('RGBA', (1, 1), 'purple'), threshold = 125):
    cols_img, rows_img = image.size
    operator_length = len(operator)
    operator = flatten_operator(operator)
    
    if (edge_image.size == (1, 1)):
        cols = cols_img - (operator_length - 1)
        rows = rows_img - (operator_length - 1)
        edge_image = Image.new('RGBA', (cols, rows), "black")
    
    for u in range(cols_img - operator_length):      # don't go off the end of cols
        for v in range(rows_img - operator_length):  # don't go off the end of rows
            subregion = get_window(u, v, operator_length, image)
            intensity = sum(s * o for s, o in zip(subregion, operator))
            if (abs(intensity) > threshold):
                edge_image.putpixel((u, v), (255, 255, 255))
    
    return edge_image

Now, with a function to aid us in our convolutionary task, lets try to do some edge detection! The above $3 \times 3$ operator that we introduced, and claimed could detect rising and falling edges, is one example of the *Robinson $3 \times 3$ operators*. These operators are recognizeable by the fact that they contain a set of 8-adjacent $1$'s and a set of 8-adjacent $-1$'s seperated only by a line of $0$'s.
>**8-Adjacency** is a description of a type of adjacency for elements arranged in a grid-like formation.

In the following code cell, we will define four **Robinson operators**. Together these operators will give our edge detection algorithm an angular resolution of $45^{\circ}$. Because our operators can detect both rising and falling edges orientated in the same direction, we can find the angular resolution with the following formula:
$$
angular\ resolution = \frac{180^{\circ}}{num\ operators}.
$$

In [ ]:
edge_operator = [[1, 0, -1],
                 [1, 0, -1],
                 [1, 0, -1]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, threshold=natural_img_threshold)

edge_operator = [[ 1,  1,  1],
                 [ 0,  0,  0],
                 [-1, -1, -1]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, edge_image=edge_image, 
                               threshold=natural_img_threshold)

edge_operator = [[ 0,  1, 1],
                 [-1,  0, 1],
                 [-1, -1, 0]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, edge_image=edge_image, 
                               threshold=natural_img_threshold)

edge_operator = [[1,  1,  0],
                 [1,  0, -1],
                 [0, -1, -1]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, edge_image=edge_image, 
                               threshold=natural_img_threshold)

In [ ]:
display(edge_image)

The relatively simple Robinson operators did a fantastic job at parsing out the edges- especially near the center of the image. This raises an interesting question, why did the edge detection algorithm perform better near the center of the image?

If we look at the vertical lines in the image, like those that outline each tower, we notice that vertical edges appear to lean toward the middle of the image. As we move further toward either side of the image, this effect becomes more distinct. Of course, in real life the edges of the tower are vertical. The apparent lean we see on these vertical lines is an affect of perspective and lens distortion.

Due to our angular resolution being $45^{\circ}$ we miss some of the edges that are further from the center. If we wanted to find these edges, we would need to increase our angular resolution.
> If this image was taken, straight on, with an orthographic camera their would be no distortion of lines away from the center.

In [ ]:
edge_operator = [[1, 0, -1],
                 [1, 0, -1],
                 [1, 0, -1]]

edge_synthetic = convolve_operator(operator=edge_operator, image=synthetic_img, threshold=synthetic_img_threshold)

edge_operator = [[ 1,  1,  1],
                 [ 0,  0,  0],
                 [-1, -1, -1]]

edge_synthetic = convolve_operator(operator=edge_operator, image=synthetic_img, threshold=synthetic_img_threshold)

edge_operator = [[ 0,  1, 1],
                 [-1,  0, 1],
                 [-1, -1, 0]]

edge_synthetic = convolve_operator(operator=edge_operator, image=synthetic_img, threshold=synthetic_img_threshold)

edge_operator = [[1,  1,  0],
                 [1,  0, -1],
                 [0, -1, -1]]

edge_synthetic = convolve_operator(operator=edge_operator, image=synthetic_img, threshold=synthetic_img_threshold)

In [ ]:
display(edge_synthetic)

The synthetic image has much stronger edges resulting in an easier decision about where an edge lies. However, we still see that the angular resolution is too small and we miss some edges. This is especially apparent on numbers with curved sides.

Let's repeat this same operation, but with a different operator. We will use the **Kirsch operators**, which take a different form, but achieve to do the same thing. Our angular resolution remains $45^{\circ}$.

In [ ]:
edge_operator = [[ 5,  5,  5],
                 [-3,  0, -3],
                 [-3, -3, -3]]

edge_natural = convolve_operator(operator=edge_operator, image=natural_img, threshold=natural_img_threshold)

edge_operator = [[5, -3, -3],
                 [5,  0, -3],
                 [5, -3, -3]]

edge_natural = convolve_operator(operator=edge_operator, image=natural_img, threshold=natural_img_threshold)

edge_operator = [[-3, -3, -3],
                 [-3,  0, -3],
                 [ 5,  5,  5]]

edge_natural = convolve_operator(operator=edge_operator, image=natural_img, threshold=natural_img_threshold)

edge_operator = [[-3, -3, 5],
                 [-3,  0, 5],
                 [-3, -3, 5]]

edge_natural = convolve_operator(operator=edge_operator, image=natural_img, threshold=natural_img_threshold)

In [ ]:
display(edge_natural)

Suprisingly, though the angular resolution of the *Kirsch operators* is the same as that of the *Robinson* operators, the *Kirsch* operators do a much better job at resolving the edges further from the center. These operators are also able to resolve some edges in the clouds. It is my belief that the larger magnitude elements in the inhibitory zone are what allow these directional operators to resolve these edges so well.

Let's see how these operators perform on our synthetic image.

In [ ]:
edge_operator = [[ 5,  5,  5],
                 [-3,  0, -3],
                 [-3, -3, -3]]

edge_synthetic = convolve_operator(operator=edge_operator, image=synthetic_img, threshold=synthetic_img_threshold)

edge_operator = [[5, -3, -3],
                 [5,  0, -3],
                 [5, -3, -3]]

edge_synthetic = convolve_operator(operator=edge_operator, image=synthetic_img, threshold=synthetic_img_threshold)

edge_operator = [[-3, -3, -3],
                 [-3,  0, -3],
                 [ 5,  5,  5]]

edge_synthetic = convolve_operator(operator=edge_operator, image=synthetic_img, threshold=synthetic_img_threshold)

edge_operator = [[-3, -3, 5],
                 [-3,  0, 5],
                 [-3, -3, 5]]

edge_synthetic = convolve_operator(operator=edge_operator, image=synthetic_img, threshold=synthetic_img_threshold)

In [ ]:
display(edge_synthetic)

Once again we find the kirsch operator performs better on the edges which fall on less orthodoxed angles and curves. Subjectively, however, I like the result of the *Robinson* operators better.
## Building an Operator
Let's build our own operator. As previously mentioned, from a stylistic standpoint, I like the *Robinson* operators better. Hence, I will riff off of those operators rather than the *Kirsch* operators. In building my version of the operator I will take some inspiration from Kirsch; in that I will allow elements not in the set $\{1, 0, -1\}$. Moreover, I want my operator to have higher angular resolution than the *Robinson's* did. To accomplish this, I will make my operators $5 \times 5$-- giving me more degrees of freedom. Finally, I will have a total of $8$ distinct operators resulting in an *angular resolution* of $22.5^{\circ}$. While this is not the recommended $10^{\circ}$, I fear that if I add too many operators the result will look very busy.

### Vertical, Horizontal, and Diagonal Edge Detectors
Below is the vertical operator that I have invisioned; following is a description of my thought process.
$$
\begin{bmatrix}
.5    &1    &0    &-1    &-.5\\
.5    &1    &0    &-1    &-.5\\
.5    &1    &0    &-1    &-.5\\
.5    &1    &0    &-1    &-.5\\
.5    &1    &0    &-1    &-.5\\
\end{bmatrix}
$$
In making my operator I thought I would simply increase the size of the *Robinson* operator to be $5 \times 5$. But this presented me a dilemma. Should I make one row of $0$'s flanked by two columns of $1$'s to left and two columns of $-1$'s to the right? Or should I make three columns of $0$'s in the center flanked by only one column of $1$'s to the left and one column of $-1$'s to the right? Naturally, I chose neither of these options. I liked the idea of having a singular column of $0$'s in the center, but I didn't want the pixels two rows away from the center column to have as much weight as the column immediately adjacent to it. Hence I taper the weight as we move away from the center column.

In order to make the horizontal edge detector, we simply rotate the vertical operator $90^{\circ}$ in either direction (here we rotate clockwise):
$$
\begin{bmatrix}
.5    &.5    &.5    &.5    &.5\\
1    &1    &1    &1    &1\\
0    &0    &0    &0    &0\\
-1    &-1    &-1    &-1    &-1\\
-.5    &-.5    &-.5    &-.5    &-.5\\
\end{bmatrix}
$$
Next we need to build an operator with the zeros along the diagonals, remembering to taper the weights as we get further from the center. In this case we will have two levels of taper, for each extra step from the center we divide by two:
$$
\begin{bmatrix}
0    &1    &.5    &.25    &.125\\
-1    &0    &1    &.5    &.25\\
-.5    &-1    &0    &1    &.5\\
-0.25    &-.5    &-1    &0    &1\\
-0.125    &-0.25    &-.5    &-1    &0\\
\end{bmatrix}\space \space
\begin{bmatrix}
.125    &.25    &.5    &1    &0\\
.25    &.5    &1    &0    &-1\\
.5    &1    &0    &-1    &-.5\\
1    &0    &-1    &-.5    &-.25\\
0    &-1    &-.5    &-.25    &-.125\\
\end{bmatrix}
$$

Before we continue to build the next four operators, let's visualize the result of these four operators.

In [ ]:

edge_operator = [[   0,  -1, -.5, -.25, -.125],
                 [   1,   0,  -1, -0.5,  -.25],
                 [  .5,   1,   0,   -1,   -.5],
                 [ .25,  .5,   1,    0,    -1],
                 [.125, .25,  .5,    1,    0]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, threshold=natural_img_threshold)

edge_operator = [[.125, .25,  .5,    1,     0],
                 [ .25,  .5,   1,    0,    -1],
                 [  .5,   1,   0,   -1,   -.5],
                 [   1,   0,  -1,  -.5,  -.25],
                 [   0,  -1, -.5, -.25, -.125]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, edge_image=edge_image, 
                               threshold=natural_img_threshold)

edge_operator = [[0.5, 0.5, 0.5, 0.5, 0.5],
                 [  1,   1,   1,   1,   1],
                 [  0,   0,   0,   0,   0],
                 [ -1,  -1,  -1,  -1,  -1],
                 [-.5, -.5, -.5, -.5, -.5]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, edge_image=edge_image, 
                               threshold=natural_img_threshold)

edge_operator = [[.5,  1,  0, -1, -.5],
                 [.5,  1,  0, -1, -.5],
                 [.5,  1,  0, -1, -.5],
                 [.5,  1,  0, -1, -.5],
                 [.5,  1,  0, -1, -.5]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, edge_image=edge_image, 
                               threshold=natural_img_threshold)

In [ ]:
display(edge_image)

In the above image, we are seeing the edges that were found using only four operators; yielding an angular resolution of $45^{\circ}$. Dispite this, the operators did a fantastic job. We see edges across the whole image, not just at the center, and even find some edges in the clouds. The detail on the door is beautiful and we can see some concentric circular arches near the base of the building.
### Edge Detectors for Intermediate Angles
As previously stated, our goal is to have an angular resolution of $22.5^{\circ}$. This means we need to build four more operators. Unfortunately, we have to give up on creating these with a clear column, row, or diagonal of zeros; perhaps this could still be accomplished with a larger operator, however, ours are only $5 \times 5$. Thus we will have to take some artistic liberties in defining this next set of operators.

Similair to the previous section, below is an illustration of a detector, after is a description of my thoughts in creating it.
$$
\begin{bmatrix}
1      &.5   &.5   &.5   &.25\\
0      &1    &1    &1    &.5\\
-1     &0    &0    &0    &1\\
-.5    &-1   &-1   &-1   &0\\
-.25   &-.5  &-.5  &-.5  &-1\\
\end{bmatrix}
$$
Following the same process as the previous set of operators, as we move away from the zero "line" we start the weights of the elements as $1$ or $-1$ and taper their weights by half for each step away from the nearest $0$ element. Because we do not have enough rows or columns to make a clear line of $0$'s we identify the angle by first creating a line of three zeros, in the center, either vertically or horizontally, and then offset the last $0$ in the direction we want to lean toward.

In the above example we start with three horizontal zeros; signifying that we want our detector to find edges which are closer to horizontal than verticle. Next we identify which direction we want to lean toward and offset the last $0$ elements according to that goal. Above we want to find angles which are $-22.5^{\circ}$ from horizontal. Hence the right most $0$ is in the last column and second to last row and the leftmost zero is in the first column and second row. To create operators for the other three intermediate directions, we can just flip this matrix according to our goal.
>It is important to note that the sum of all elements is still zero.

In [ ]:

edge_operator = [[   1,  .5,  .5,  .5, 0.25],
                 [   0,   1,   1,   1,  0.5],
                 [  -1,   0,   0,   0,    1],
                 [ -.5,  -1,  -1,  -1,    0],
                 [-.25, -.5, -.5, -.5,   -1]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, edge_image=edge_image, 
                               threshold=natural_img_threshold)

edge_operator = [[0.25,  .5,  .5,  .5,    1],
                 [  .5,   1,   1,   1,    0],
                 [   1,   0,   0,   0,   -1],
                 [   0,  -1,  -1,  -1,  -.5],
                 [  -1, -.5, -.5, -.5, -.25]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, edge_image=edge_image, 
                               threshold=natural_img_threshold)

edge_operator = [[0.25, .5,  1,  0,    -1],
                 [  .5,  1,  0,  -1,  -.5],
                 [  .5,  1,  0,  -1,  -.5],
                 [  .5,  1,  0,  -1,  -.5],
                 [   1,  0, -1, -.5, -.25]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, edge_image=edge_image, 
                               threshold=natural_img_threshold)

edge_operator = [[  -1,   0,  1, .5, .25],
                 [ -.5,  -1,  0,  1,  .5],
                 [ -.5,  -1,  0,  1,  .5],
                 [ -.5,  -1,  0,  1,  .5],
                 [-.25, -.5, -1,  0,   1]]

edge_image = convolve_operator(operator=edge_operator, image=natural_img, edge_image=edge_image, 
                               threshold=natural_img_threshold)

In [ ]:
display(edge_image)

Again, I am very pleased with how this image has turned out. We can now see even finer detail in both the cathedral and the clouds. The series of smaller arches have also become very clear. Each brick is clearly defined and we even find imperfections within them.

Our last task is to repeat this process for our synthetic image.

In [ ]:

edge_operator = [[   0,  -1, -.5, -.25, -.125],
                 [   1,   0,  -1, -0.5,  -.25],
                 [  .5,   1,   0,   -1,   -.5],
                 [ .25,  .5,   1,    0,    -1],
                 [.125, .25,  .5,    1,    0]]

edge_image = convolve_operator(operator=edge_operator, image=synthetic_img, threshold=synthetic_img_threshold)

edge_operator = [[.125, .25,  .5,    1,     0],
                 [ .25,  .5,   1,    0,    -1],
                 [  .5,   1,   0,   -1,   -.5],
                 [   1,   0,  -1,  -.5,  -.25],
                 [   0,  -1, -.5, -.25, -.125]]

edge_image = convolve_operator(operator=edge_operator, image=synthetic_img, edge_image=edge_image, 
                               threshold=synthetic_img_threshold)

edge_operator = [[0.5, 0.5, 0.5, 0.5, 0.5],
                 [  1,   1,   1,   1,   1],
                 [  0,   0,   0,   0,   0],
                 [ -1,  -1,  -1,  -1,  -1],
                 [-.5, -.5, -.5, -.5, -.5]]

edge_image = convolve_operator(operator=edge_operator, image=synthetic_img, edge_image=edge_image, 
                               threshold=synthetic_img_threshold)

edge_operator = [[.5,  1,  0, -1, -.5],
                 [.5,  1,  0, -1, -.5],
                 [.5,  1,  0, -1, -.5],
                 [.5,  1,  0, -1, -.5],
                 [.5,  1,  0, -1, -.5]]

edge_image = convolve_operator(operator=edge_operator, image=synthetic_img, edge_image=edge_image, 
                               threshold=synthetic_img_threshold)

edge_operator = [[   1,  .5,  .5,  .5, 0.25],
                 [   0,   1,   1,   1,  0.5],
                 [  -1,   0,   0,   0,    1],
                 [ -.5,  -1,  -1,  -1,    0],
                 [-.25, -.5, -.5, -.5,   -1]]

edge_image = convolve_operator(operator=edge_operator, image=synthetic_img, edge_image=edge_image, 
                               threshold=synthetic_img_threshold)

edge_operator = [[0.25,  .5,  .5,  .5,    1],
                 [  .5,   1,   1,   1,    0],
                 [   1,   0,   0,   0,   -1],
                 [   0,  -1,  -1,  -1,  -.5],
                 [  -1, -.5, -.5, -.5, -.25]]

edge_image = convolve_operator(operator=edge_operator, image=synthetic_img, edge_image=edge_image, 
                               threshold=synthetic_img_threshold)

edge_operator = [[0.25, .5,  1,  0,    -1],
                 [  .5,  1,  0,  -1,  -.5],
                 [  .5,  1,  0,  -1,  -.5],
                 [  .5,  1,  0,  -1,  -.5],
                 [   1,  0, -1, -.5, -.25]]

edge_image = convolve_operator(operator=edge_operator, image=synthetic_img, edge_image=edge_image, 
                               threshold=synthetic_img_threshold)

edge_operator = [[  -1,   0,  1, .5, .25],
                 [ -.5,  -1,  0,  1,  .5],
                 [ -.5,  -1,  0,  1,  .5],
                 [ -.5,  -1,  0,  1,  .5],
                 [-.25, -.5, -1,  0,   1]]

edge_image = convolve_operator(operator=edge_operator, image=synthetic_img, edge_image=edge_image, 
                               threshold=synthetic_img_threshold)

In [ ]:
display(edge_image)

The set of edge operators that we have created also works very well for our synthetic image. We no longer miss the rounded sides of the numbers nor any flat side that sits at an odd angle.